# Проверяем файлы в кластере и устанавливаем библиотеки

In [1]:
!pip install pyspark

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████▏   | 278.9 MB 105.0 MB/s eta 0:00:01 |▎                               | 2.7 MB 2.6 MB/s eta 0:02:00| 157.1 MB 115.7 MB/s eta 0:00:02

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



     |████████████████████████████████| 316.9 MB 2.6 kB/s 
     |████████████████████████████████| 200 kB 85.1 MB/s eta 0:00:01
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425367 sha256=f2ad5f5a5413f09305c1de3d7e64bd9971d18a6509d80b831ff695a009449dc7
  Stored in directory: /home/ubuntu/.cache/pip/wheels/a6/ce/f9/17d82c92f044018df2fe30af63ac043447720d5b2cee39b40f
Successfully built pyspark


In [2]:
!pip install findspark

Defaulting to user installation because normal site-packages is not writeable


In [1]:
import findspark
findspark.init()
findspark.find()

'/usr/lib/spark'

In [2]:
!hdfs dfs  -ls /user/hive/warehouse/froms3

Found 40 items
-rw-r--r--   1 ubuntu hadoop 2807409271 2024-01-14 09:40 /user/hive/warehouse/froms3/2019-08-22.txt
-rw-r--r--   1 ubuntu hadoop 2854479008 2024-01-14 09:38 /user/hive/warehouse/froms3/2019-09-21.txt
-rw-r--r--   1 ubuntu hadoop 2895460543 2024-01-14 09:38 /user/hive/warehouse/froms3/2019-10-21.txt
-rw-r--r--   1 ubuntu hadoop 2939120942 2024-01-14 09:40 /user/hive/warehouse/froms3/2019-11-20.txt
-rw-r--r--   1 ubuntu hadoop 2995462277 2024-01-14 09:38 /user/hive/warehouse/froms3/2019-12-20.txt
-rw-r--r--   1 ubuntu hadoop 2994906767 2024-01-14 09:37 /user/hive/warehouse/froms3/2020-01-19.txt
-rw-r--r--   1 ubuntu hadoop 2995431240 2024-01-14 09:40 /user/hive/warehouse/froms3/2020-02-18.txt
-rw-r--r--   1 ubuntu hadoop 2995176166 2024-01-14 09:38 /user/hive/warehouse/froms3/2020-03-19.txt
-rw-r--r--   1 ubuntu hadoop 2996034632 2024-01-14 09:41 /user/hive/warehouse/froms3/2020-04-18.txt
-rw-r--r--   1 ubuntu hadoop 2995666965 2024-01-14 09:40 /user/hive/warehouse/froms3/

In [3]:
path = '/user/hive/warehouse/froms3'

In [4]:
import pyspark
app_name = "myApp" # Ваше имя
spark_ui_port = 4040 # Порт для spark ui

spark = (
    pyspark.sql.SparkSession.builder
        .appName(app_name) # имя приложения ? Нужно для отслеживания таски выполнения
        .config("spark.executor.cores", "4")        
        .config("spark.executor.memory", "4g") # Executor просее. Ориенир для потребления помаяти. 
        .config("spark.executor.instances", "6")    
        .config("spark.default.parallelism", "48")
        .config("spark.driver.memory", "4g") # Main процесс
        .config("spark.ui.port", spark_ui_port)
        .getOrCreate()
)
spark.conf.set('spark.sql.repl.eagerEval.enabled', True)  # to pretty print pyspark.DataFrame in jupyter

In [5]:
from pyspark.sql.types import StructType, StructField, IntegerType, FloatType, TimestampType, StringType

schema = StructType([
    StructField("tranaction_id", IntegerType(), True),
    StructField("tx_datetime", TimestampType(), True),
    StructField("customer_id", IntegerType(), True),
    StructField("terminal_id", IntegerType(), True),
    StructField("tx_amount", FloatType(), True),
    StructField("tx_time_seconds", IntegerType(), True),
    StructField("tx_time_days", IntegerType(), True),
    StructField("tx_fraud", IntegerType(), True),
    StructField("tx_fraud_scenario", IntegerType(), True)])

#data = spark.read.text('/user/hive/warehouse/2019-09-21.txt')

df = spark.read.csv(path, 
                    header=False, schema=schema, comment = "#")


In [6]:
import pandas as pd

print('Data overview')
df.printSchema()
print('Columns overview')
pd.DataFrame(df.dtypes, columns = ['Column Name','Data type'])

Data overview
root
 |-- tranaction_id: integer (nullable = true)
 |-- tx_datetime: timestamp (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- terminal_id: integer (nullable = true)
 |-- tx_amount: float (nullable = true)
 |-- tx_time_seconds: integer (nullable = true)
 |-- tx_time_days: integer (nullable = true)
 |-- tx_fraud: integer (nullable = true)
 |-- tx_fraud_scenario: integer (nullable = true)

Columns overview


,Column Name,Data type
0,tranaction_id,int
1,tx_datetime,timestamp
2,customer_id,int
3,terminal_id,int
4,tx_amount,float
5,tx_time_seconds,int
6,tx_time_days,int
7,tx_fraud,int
8,tx_fraud_scenario,int


In [7]:
df.write.parquet("/user/hive/warehouse/output_preliminary.parquet")

In [8]:
!hdfs dfs  -ls /user/hive/warehouse

Found 2 items
drwxrwxrwx   - ubuntu hadoop          0 2024-01-14 09:41 /user/hive/warehouse/froms3
drwxr-xr-x   - ubuntu hadoop          0 2024-01-14 10:16 /user/hive/warehouse/output_preliminary.parquet


In [9]:
df_p = spark.read.parquet('/user/hive/warehouse/output_preliminary.parquet')

In [10]:
df_p.describe().toPandas().T

,0,1,2,3,4
summary,count,mean,stddev,min,max
tranaction_id,1879794138,9.398958918339887E8,5.42649141382195E8,0,1879791584
customer_id,1879794138,500416.7652254882,288588.1304047357,-999999,999999
terminal_id,1879753826,24933.921642023033,1478655.8994933835,0,89518096
tx_amount,1879794138,54.231436693500704,41.286941330176944,0.0,16539.04
tx_time_seconds,1879794138,5.184045645780446E7,2.9929841937784676E7,0,103680000
tx_time_days,1879794138,599.5075215422339,346.4094747447377,0,1199
tx_fraud,1879794138,0.05922921385341611,0.23605320186213166,0,1
tx_fraud_scenario,1879794138,0.11930446077388501,0.47748996092105045,0,3


Потенциальные проблемы в датасете:

1. могут быть пустые  или нулевые значения (например, по terminal_id - кол-во записей меньше; по tx_amount минимальное значение 0)
2. могут быть дубликаты (например, номер транзакции должен быть уникальный)
3. могут быть непонятные и неправильные id (например, очень странно видеть id клиента отрицательным; также имеет смысл проверить terminal_id - действительно он может быть таким)
4. проверить, что tx_fraud либо 0, либо 1 и что tx_fraud_scenario из списка [0,1,2,3]

**Комментарий**: не совсем понятна сущность фичей tx_time_seconds и tx_time_days, поэтому в рамках данного анализа их рассматривать не буду.

# 1. Проверка на пустые и нулевые значения

In [11]:
from pyspark.sql.functions import lit, desc, col, size, array_contains\
, isnan, udf, hour, array_min, array_max, countDistinct, max, min

import numpy as np

In [12]:
spark.conf.set("spark.sql.legacy.timeParserPolicy","LEGACY")

# Предположим, что 0 может быть id как клиента, так и терминала
string_columns = ['tranaction_id', 'customer_id', 'terminal_id', 'tx_datetime']
not_to_review = ['tx_fraud', 'tx_fraud_scenario']

missing_values = {} 
for index, column in enumerate(df_p.columns):
    if column in string_columns:    # check string columns with None and Null values
        missing_count = df_p.filter(col(column).eqNullSafe(None) | col(column).isNull()).count()
        missing_values.update({column:missing_count})
    elif column not in not_to_review:
        missing_count = df_p.where(col(column).isin([0.0, None, np.nan])).count()
        missing_values.update({column:missing_count})

missing_df = pd.DataFrame.from_dict([missing_values])
missing_df

,tranaction_id,tx_datetime,customer_id,terminal_id,tx_amount,tx_time_seconds,tx_time_days
0,0,3805,0,40312,35260,3,1565894


## Вывод:
В некторых столбцах есть пропуски и нулевые значения, будем исключать такие строки при предобработке данных. Комментарий по поводу суммы транзакции - будем считать, что если сумма нулевая, то она ошибочна и не будет включаться в анализ. Очевидно, что здесь нужно больше данных для более точного решения.
Общее кол-во таких транзакций не значительно и их удаление без корректировки не сильно повлияет на обучение моделей в будущем.


# 2. Проверка на дубликаты

In [13]:
duplicated = df_p.count() - df_p.select('tranaction_id').distinct().count()
duplicated

2553

In [14]:
ids_used = df_p.select(max(df_p.tranaction_id).alias('max_value')).first().max_value - df_p.select(min(df_p.tranaction_id).alias('min_value')).first().min_value
unique_ids = df_p.select('tranaction_id').distinct().count()
print(f'Compared unique ids = {unique_ids} with sequences used = {ids_used}')

Compared unique ids = 1879791585 with sequences used = 1879791584


In [15]:
ids_used - unique_ids

-1

## Выводы.

Итак, есть дубликаты ids для транзакций. Так как все-таки значения должны быть уникальными, то при обработке нужно дубликаты удалить. Отмечу, что таких дубликатов немного и удаление не повлияет на обучение моделей в будущем. 
Проверка на последовательность используемых ids показала, что не все id используются последовательно. Так как разница не совпадает с кол-вом дублиактов, то некторые id не используются. На данном примере оставлю это вариант, в реальном кейсе я бы посмотрела, почему некторые id выпали. 

# 3. Проверка на правильность id

In [16]:
to_review = ['customer_id', 'terminal_id']
for item in to_review:
    sort = df_p.select(item).distinct().orderBy(item)
    print(f'Head for {item}:\n{sort.head(10)}')
    print(f'Tail for {item}:\n{sort.tail(10)}')
    print('===================================')

Head for customer_id:
[Row(customer_id=-999999), Row(customer_id=0), Row(customer_id=1), Row(customer_id=2), Row(customer_id=3), Row(customer_id=4), Row(customer_id=5), Row(customer_id=6), Row(customer_id=7), Row(customer_id=8)]
Tail for customer_id:
[Row(customer_id=999990), Row(customer_id=999991), Row(customer_id=999992), Row(customer_id=999993), Row(customer_id=999994), Row(customer_id=999995), Row(customer_id=999996), Row(customer_id=999997), Row(customer_id=999998), Row(customer_id=999999)]
Head for terminal_id:
[Row(terminal_id=None), Row(terminal_id=0), Row(terminal_id=1), Row(terminal_id=2), Row(terminal_id=3), Row(terminal_id=4), Row(terminal_id=5), Row(terminal_id=6), Row(terminal_id=7), Row(terminal_id=8)]
Tail for terminal_id:
[Row(terminal_id=997), Row(terminal_id=998), Row(terminal_id=999), Row(terminal_id=21861), Row(terminal_id=22129), Row(terminal_id=22225), Row(terminal_id=360740), Row(terminal_id=436002), Row(terminal_id=618230), Row(terminal_id=89518096)]


In [17]:
strange_customer_ids = (
    df_p
    .filter(df_p['customer_id'] < 0)
)
strange_customer_ids.count()

14623

In [18]:
strange_terminal_ids = (
    df_p
    .filter(df_p['terminal_id'] > 999)
)
strange_terminal_ids.count()

575157

## Выводы.
Проверка показала, что нужно будет удалить строки с отрицательным customer_id, так как это скорее всего ошибки. Кол-во таких строк незначительно, поэтому легче удалить, чем исследовать причину ошибки. Хотя в реальной ситуации, я бы провела мини расследования для понимания причины. Может быть так обозначаются ушедшие клиенты и т.д.

Также удалить строки, где terminal_id > 999, так как это скорее всего ошибка. Здесь кол-во записей с такими странными номерами больше, лучше бы провести небольшое расследование. Для учебного примера будем удалять такие строки.

# 4. Проверка на значения tx_fraud и tx_fraud_scenario


In [19]:
outcome_count = df_p.groupBy('tx_fraud').count().toPandas()
outcome_count

,tx_fraud,count
0,1,111338729
1,0,1768455409


In [20]:
scenario_count = df_p.groupBy('tx_fraud_scenario').count().toPandas()
scenario_count

,tx_fraud_scenario,count
0,1,1018093
1,3,2608461
2,2,107712175
3,0,1768455409


## Выводы
Итак, обе фичи не содержат ошибок и не требуют дополнительной проверки.

In [21]:
spark.stop()